In [1]:
import pandas as pd 
import dask.dataframe as dd
import os
import time
import gc
import numpy as np
train_path = '../input/train.csv'
print(train_path)

../input/train.csv


In [2]:
os.getcwd()
nrows=6_000_000
# def train(nrows=None):
# Set columns to most suitable type to optimize for memory usage
t0 = time.time()
traintypes = {'key':'str',
              'fare_amount': 'float32',
     'pickup_datetime': 'str', 
     'pickup_longitude': 'float32',
     'pickup_latitude': 'float32',
     'dropoff_longitude': 'float32',
     'dropoff_latitude': 'float32',
     'passenger_count': 'uint8'}
train = pd.read_csv('../03 Google Taxi Fare Prediction/input/train.csv',nrows=nrows,dtype=traintypes,usecols=traintypes.keys())
train.dropna(inplace=True)
testtypes = {'key':'str',
             'pickup_datetime': 'str', 
     'pickup_longitude': 'float32',
     'pickup_latitude': 'float32',
     'dropoff_longitude': 'float32',
     'dropoff_latitude': 'float32',
     'passenger_count': 'uint8'}
test = pd.read_csv('../03 Google Taxi Fare Prediction/input/test.csv',nrows=nrows,dtype=traintypes,usecols=testtypes.keys())
print("the reading time for df set is {:.0f} s".format((time.time() - t0)))
df = pd.concat([train,test],axis=0,sort=False)
print("the training shape is {}".format(train.shape))
print("the df shape is {}".format(df.shape))
display(df.head())
display(df.tail())
print(df.info())
del train,test
gc.collect();

the reading time for df set is 13 s
the training shape is (5999961, 8)
the df shape is (6009875, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844315,40.721317,-73.841614,40.712276,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016045,40.711304,-73.979271,40.782005,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982735,40.761269,-73.991241,40.750561,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987129,40.733143,-73.991570,40.758091,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968094,40.768009,-73.956657,40.783764,1


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
9909,2015-05-10 12:37:51.0000002,NaN,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51.0000001,NaN,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15.0000001,NaN,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19.0000005,NaN,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6
9913,2015-01-18 14:06:23.0000006,NaN,2015-01-18 14:06:23 UTC,-73.988022,40.754070,-74.000282,40.759220,6


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6009875 entries, 0 to 9913
Data columns (total 8 columns):
key                  object
fare_amount          float32
pickup_datetime      object
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count      uint8
dtypes: float32(5), object(2), uint8(1)
memory usage: 257.9+ MB
None


In [3]:
#some EDA
display(df.describe())
print("There have somebody who takes a ride with 93_963 $ !!! Seem pretty wierd right ?")
display(df.fare_amount.sort_values(ascending=False).head(10))
print("Only little missing data out there")
display(df.isna().mean())

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.999961e+06,6.009875e+06,6.009875e+06,6.009875e+06,6.009875e+06,6.009875e+06
mean,1.132846e+01,-6.985543e+01,4.263308e+01,-6.985470e+01,4.262899e+01,1.684634e+00
std,9.764237e+00,1.284012e+01,9.266454e+00,1.290134e+01,9.596098e+00,1.327712e+00
min,-1.000000e+02,-3.426609e+03,-3.488080e+03,-3.412653e+03,-3.488080e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073491e+01,-7.399139e+01,4.073404e+01,1.000000e+00
50%,8.500000e+00,-7.398182e+01,4.075263e+01,-7.398016e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396711e+01,4.076712e+01,-7.396368e+01,4.076811e+01,2.000000e+00
max,1.273310e+03,3.439426e+03,3.344459e+03,3.457622e+03,3.345917e+03,2.080000e+02


There have somebody who takes a ride with 93_963 $ !!! Seem pretty wierd right ?


1144706    1273.310059
4894706     952.000000
3723759     698.159973
3206977     542.830017
3316931     500.000000
101885      500.000000
5388368     499.000000
247671      495.000000
1215047     488.000000
329010      450.000000
Name: fare_amount, dtype: float32

Only little missing data out there


key                  0.00000
fare_amount          0.00165
pickup_datetime      0.00000
pickup_longitude     0.00000
pickup_latitude      0.00000
dropoff_longitude    0.00000
dropoff_latitude     0.00000
passenger_count      0.00000
dtype: float64

let's calculate the distance from latitude and longtitude

In [4]:
df.loc[~df['fare_amount'].isna()].isna().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [5]:
df['diff_long'] = (df['pickup_longitude'] - df['dropoff_longitude']).abs()
df['diff_lat'] = (df['pickup_latitude'] - df['dropoff_latitude']).abs()
df['displacement'] = np.sqrt((df['diff_long'])**2 + (df['diff_lat'])**2)
# display(df.head())

In [6]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer

In [ ]:
# %%time
import lightgbm as lgb
from sklearn.model_selection import KFold
df_train = df[df['fare_amount'].notnull()].drop(columns=['fare_amount']).select_dtypes('number')
df_label = df[df['fare_amount'].notnull()]['fare_amount'].copy()

# train test split
# train_features,valid_feature , train_label, valid_label = train_test_split(df_train, df_label, test_size = 0.3,random_state=42);
    
regressor = lgb.LGBMRegressor(nthread=1,
         n_estimators=10000,
         learning_rate=0.02,
         num_leaves=34, 
         colsample_bytree=0.9497036,
         subsample=0.8715623,
         max_depth=8,
         reg_alpha=0.041545473,
         reg_lambda=0.0735294,
         min_split_gain=0.0222415,
         min_child_weight=39.3259775,
         silent=-1,
         verbose=-1) # from aguiar

#kfold 
RMSE = [];
folds = KFold(n_splits=3,shuffle = True, random_state=42)
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(df_train, df_label)):
    
    train_feature, train_label = df_train.iloc[train_idx], df_label.iloc[train_idx]
    valid_feature, valid_label = df_train.iloc[valid_idx], df_label.iloc[valid_idx]
#     import pdb; pdb.set_trace()
    regressor.fit(train_feature,train_label,
                  eval_set=[(train_feature, train_label), (valid_feature, valid_label)],
                  early_stopping_rounds = 200,verbose=50,eval_metric='l2')
    valid_pred = regressor.predict(valid_feature)

    RMSE.append(np.sqrt(mean_squared_error(valid_label,valid_pred)))
    print("finish fold {}".format(n_fold+1))
    

In [35]:
%%time
np.mean(RMSE)

Wall time: 0 ns


4.601772573064144

In [21]:
test_df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,diff_long,diff_lat,displacement
0,NaN,-73.973320,40.763805,-73.981430,40.743835,1,0.008110,0.019970,0.021554
1,NaN,-73.986862,40.719383,-73.998886,40.739201,1,0.012024,0.019817,0.023180
2,NaN,-73.982521,40.751259,-73.979652,40.746140,1,0.002869,0.005119,0.005868
3,NaN,-73.981163,40.767807,-73.990448,40.751637,1,0.009285,0.016171,0.018647
4,NaN,-73.966049,40.789776,-73.988564,40.744427,1,0.022514,0.045349,0.050630


In [36]:
%%time
test_df = df[df['fare_amount'].isnull()].drop(columns='fare_amount').select_dtypes('number')
test_df_pred = regressor.predict(test_df)

Wall time: 5.04 s


In [34]:
print(regressor.feature_importances_/np.sum(regressor.feature_importances_)*100)
print(train_feature.columns)

[15.49248107 16.45173185 15.59451828 15.98751307  2.44283138 11.9156223
 11.73983543 10.37546662]
Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'diff_long', 'diff_lat',
       'displacement'],
      dtype='object')


In [ ]:
# Divide in training/validation and test data
train_df = df[df['fare_amount'].notnull()]
test_df = df[df['fare_amount'].isnull()]
print("Starting. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
# del df
gc.collect()

train_df_label = train_df.pop("fare_amount")
train_df_feature = train_df.select_dtypes("number")
test_df_feature = test_df.drop(columns='fare_amount').select_dtypes('number')

#estimator 
alpha = np.logspace(-3,3,7)
lasso = linear_model.LassoCV(alphas = alpha, cv = 5, verbose = 0);
lasso

# baseline model 
t0 = time.time()
lasso = lasso.fit(train_df_feature,train_df_label)
print("the traning time is {:.0f} s".format((time.time() - t0)))
test_df_pred = lasso.predict(test_df_feature)

In [ ]:
test_df_pred[:5]

In [27]:
submission = pd.concat([df[df['fare_amount'].isnull()]['key'],pd.DataFrame(test_df_pred,columns=['fare_amount'])],axis=1)
print(submission.head())
submission.to_csv("lightgbm.csv",index_label=False,index=False)

                           key  fare_amount
0  2015-01-27 13:08:24.0000002     8.518349
1  2015-01-27 13:08:24.0000003     8.738415
2  2011-10-08 11:53:44.0000002     4.814409
3  2012-12-01 21:12:12.0000002     8.149880
4  2012-12-01 21:12:12.0000003    14.842281


In [28]:
submission.shape

(9914, 2)